In [25]:
import cv2 as cv
import numpy as np

video = cv.VideoCapture( 'Camera_Calibration.mp4' )

if video.isOpened() == False:
    exit()

fps = 50
wait_msec = int( 1000 / fps )
width = int(video.get(3))
height = int(video.get(4))
    
board_cellsize = 2.5
board_pattern = (8,6)

# 1204 0 599
# 0 1215 349
# 0 0 1

K = np.array([[1204.5, 0, 599],
              [0, 1215, 349],
              [0, 0, 1]])
dist_coeff = np.array([0.039, 0.159, -0.000657, 0.00735, -0.693])

# Prepare a 3D box for simple AR
#box_lower = board_cellsize * np.array([[4, 2, 0], [5, 2, 0], [5, 4, 0], [4, 4, 0]])
#box_upper = board_cellsize * np.array([[4, 2, -1], [5, 2, -1], [5, 4, -1], [4, 4, -1]])


pyramid_down = board_cellsize * np.array([[1, 1, 0], [1, 5, 0], [5, 5, 0], [5, 1, 0]], dtype=np.float32)
pyramid_up = board_cellsize * np.array([3, 3, -6])

# Prepare 3D points on a chessboard
obj_points = board_cellsize * np.array([[c, r, 0] for r in range(board_pattern[1]) for c in range(board_pattern[0])])

while True:
# Read an image from the video
    valid, img = video.read()
    key = cv.waitKey(wait_msec)
        
    if key == 27:
        break
    
    if not valid:
        break

    complete, img_points = cv.findChessboardCorners(img, board_pattern)
    
    if complete:
        ret, rvec, tvec = cv.solvePnP(obj_points, img_points, K, dist_coeff)
        
        down_points, _ = cv.projectPoints(pyramid_down, rvec, tvec, K, dist_coeff)
        up_point, _ = cv.projectPoints(pyramid_up, rvec, tvec, K, dist_coeff)
        
        pt3 = up_point.ravel().astype(np.int32)
        
        pt1 = tuple(down_points[0].ravel().astype(np.int32))
        pt2 = tuple(down_points[1].ravel().astype(np.int32))
        cv.line(img, pt1, pt2, (0, 255, 0), 2)
        cv.line(img, pt1, pt3, (0, 255, 0), 2)
        pt1 = tuple(down_points[1].ravel().astype(np.int32))
        pt2 = tuple(down_points[2].ravel().astype(np.int32))
        cv.line(img, pt1, pt2, (0, 255, 0), 2)
        cv.line(img, pt1, pt3, (0, 255, 0), 2)
        pt1 = tuple(down_points[2].ravel().astype(np.int32))
        pt2 = tuple(down_points[3].ravel().astype(np.int32))
        cv.line(img, pt1, pt2, (0, 255, 0), 2)
        cv.line(img, pt1, pt3, (0, 255, 0), 2)
        pt1 = tuple(down_points[3].ravel().astype(np.int32))
        pt2 = tuple(down_points[0].ravel().astype(np.int32))
        cv.line(img, pt1, pt2, (0, 255, 0), 2)
        cv.line(img, pt1, pt3, (0, 255, 0), 2)
        
        #for b, t in zip(line_lower, line_upper):
            #cv.line(img, np.int32(b.flatten()), np.int32(t.flatten()), (0, 255, 0), 2)
        
        
        #line_lower, _ = cv.projectPoints(box_lower, rvec, tvec, K, dist_coeff)
        #line_upper, _ = cv.projectPoints(box_upper, rvec, tvec, K, dist_coeff)
        #cv.polylines(img, [np.int32(line_lower)], True, (255, 0, 0), 2)
        #cv.polylines(img, [np.int32(line_upper)], True, (0, 0, 255), 2)
        
        #for b, t in zip(line_lower, line_upper):
            #cv.line(img, np.int32(b.flatten()), np.int32(t.flatten()), (0, 255, 0), 2)
        
        # Print the camera position
        R, _ = cv.Rodrigues(rvec) # Alternative) `scipy.spatial.transform.Rotation`
        p = (-R.T @ tvec).flatten()
        info = f'XYZ: [{p[0]:.3f} {p[1]:.3f} {p[2]:.3f}]'
        cv.putText(img, info, (10, 25), cv.FONT_HERSHEY_DUPLEX, 0.6, (0, 255, 0))
        
        cv.imshow('Video Player', cv.resize(img, (width, height))   )
    else:
        video.release()
        cv.destroyAllWindows()
    
    
video.release()
cv.destroyAllWindows()